In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances 

In [2]:
#Load data
df = pd.read_csv('UserRatings1.csv')

In [3]:
df.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


In [4]:
data = df.transpose().fillna(0)

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
JokeId,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,...,90.00,91.00,92.00,93.00,94.00,95.00,96.00,97.00,98.00,99.00
User1,5.10,4.90,1.75,-4.17,5.15,1.75,4.76,3.30,-2.57,-1.41,...,5.34,-4.61,3.59,7.18,0.92,6.31,-4.95,-0.19,3.25,4.37
User2,-8.79,-0.87,1.99,-4.61,5.39,-0.78,1.60,1.07,-8.69,-4.66,...,3.59,1.21,2.86,-0.05,-1.75,-1.02,-0.97,4.13,-1.84,2.96
User3,-3.50,-2.91,-2.18,-0.10,7.52,1.26,-5.39,1.50,-8.40,4.37,...,1.84,-4.03,-1.41,1.65,-3.79,3.98,-6.46,-6.89,-2.33,-7.38
User4,7.14,-3.88,-3.06,0.05,6.26,6.65,-7.52,7.28,-5.15,-7.14,...,-4.47,6.36,4.71,-5.19,6.26,3.93,-2.57,1.07,2.33,-0.34


In [6]:
#clean
data = data.iloc[1:,:]

In [7]:
user_similarity = pairwise_distances(data, metric='cosine')

In [8]:
#Predict user - user using top30
def predict_user_user(train_matrix, user_similarity, n_similar=30):
    similar_n = user_similarity.argsort()[:,-n_similar:][:,::-1]
    n_users_base = train_matrix.shape[0]
    n_items_base = train_matrix.shape[1]
    pred = np.zeros((n_users_base,n_items_base))
    for i,users in enumerate(similar_n):
        similar_users_indexes = users
        similarity_n = user_similarity[i,similar_users_indexes]
        matrix_n = train_matrix[similar_users_indexes,:]
        rated_items = similarity_n[:,np.newaxis].T.dot(matrix_n - matrix_n.mean(axis=1)[:,np.newaxis])/ similarity_n.sum()
        pred[i,:]  = rated_items
    return pred

In [9]:
#Use top 50 and then add average rating, which was used to normalize in predict_user_user
data_np = data.values
predictions = predict_user_user(data_np,user_similarity, 50) + data.mean(axis=1)[:, np.newaxis]


In [17]:
print(predictions.shape)

(36710, 100)


In [16]:
print(data.shape)

(36710, 100)


In [24]:
#Find error in each predictions
e = np.abs(np.subtract(predictions,data))

In [31]:
#Find average error
tot = e.shape[0] * e.shape[1]
print((e.sum().sum())/tot)

4.705225201117708


In [36]:
#Item similarity
item_similarity = pairwise_distances(data_np.T, metric = 'cosine')
item_similarity.shape

(100, 100)

In [38]:
#Define function to prediction using item-item
def predict_item_item(train_matrix, item_similarity, n_similar=30):
    n_users_base = train_matrix.shape[0]
    n_items_base = train_matrix.shape[1]
    similar_n = item_similarity.argsort()[:,-n_similar:][:,::-1]
    print('similar_n shape: ', similar_n.shape)
    pred = np.zeros((n_users_base,n_items_base))
    
    for i,items in enumerate(similar_n):
        similar_items_indexes = items
        similarity_n = item_similarity[i,similar_items_indexes]
        matrix_n = train_matrix[:,similar_items_indexes]
        rated_items = matrix_n.dot(similarity_n)/similarity_n.sum()
        pred[:,i]  = rated_items
    return pred


In [39]:
#Find predictions
predictions = predict_item_item(data_np,item_similarity,50)
print('predictions shape ',predictions.shape)


similar_n shape:  (100, 50)
predictions shape  (36710, 100)


In [40]:
#Find error in each predictions
e = np.abs(np.subtract(predictions,data))

In [41]:
#Find average error
tot = e.shape[0] * e.shape[1]
print((e.sum().sum())/tot)

3.871993759236794
